In [53]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = '2g', ip = "127.0.0.1", port = 54321)


Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,59 mins 23 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_frikk_emdefq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.086 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [54]:
import pandas as pd
import os
import numpy as np

In [55]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)

folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)

In [56]:
#Get target
target = pd.read_csv("../data/stores_train.csv")
target = target["revenue"].values

target = np.log1p(target)

In [57]:
features_train["revenue"] = target

In [58]:
features_train.to_csv("h2o_automl_2_features_csv/features_train.csv", index=False)

features_test.to_csv("h2o_automl_2_features_csv/features_test.csv", index=False)

In [59]:
df_train = h2o.import_file("h2o_automl_2_features_csv/features_train.csv")
df_test = h2o.import_file("h2o_automl_2_features_csv/features_test.csv")

df_test.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:8577
Cols:18

,average_income_per_grunnkrets,numberOfBusstop,categorical_chain_name,categorical_mall_name,categorical_plaace,categorical_sales_channel_name,categorical_street,distance_store_busstop,distance_to_competitor,has_mall_or_not,number_adults_grunnkrets,children_number,couple_number,features,person_number,singles_number,area_grunnkrets,feature
type,real,int,enum,enum,enum,enum,enum,real,real,int,real,real,real,int,real,real,real,int
mins,0.0,0.0,,,,,,2.834648843907257,0.0,0.0,1.0,0.0,0.0,0.0,66.0,0.0,0.0034185853132582,0.0
mean,559440.6715634838,15.12277019937041,,,,,,363.42705370954155,1049.381980845635,0.17407018771132096,297.2205535559141,129.97669228861892,170.24356412366285,0.6181648595079865,734.0515745102902,262.1103796873163,0.7512256878394341,0.6044071353620147
maxs,799720.0,72.0,,,,,,40354.63748683241,128872.47008953274,1.0,2567.0,1789.0,1012.0,18.0,5949.0,2002.0,14.995158557421975,13.0
sigma,68128.77375037978,12.834269924280061,,,,,,1223.2054558757015,4092.954112855524,0.3791919324813381,270.2875134203443,137.68181653627445,145.78886964383148,1.8292455318578489,585.2326052568449,234.67556085195426,1.119024389594291,1.4938756118063603
zeros,27,377,,,,,,0,2144,7084,0,33,203,6880,0,21,0,6403
missing,0,0,6099,7084,0,0,1239,0,0,0,0,0,0,0,0,0,0,0
0,551600.0,21.0,,,1.1.1.0,Hamburger restaurants,STRÃ˜MSVEIEN,25.63006058192599,70.08576024451624,0.0,493.0,224.0,176.0,0.0,970.0,391.0,0.0570274487828266,0.0
1,531240.0,11.0,BURGER KING,,1.1.1.0,Hamburger restaurants,MYREN,111.73618940669482,177.7104392523965,0.0,297.22055355591414,129.9766922886189,0.0,0.0,734.0515745102902,3.0,0.1659929866554556,0.0
2,523300.0,11.0,BURGER KING,Stovner Senter,1.1.1.0,Hamburger restaurants,STOVNER SENTER,404.825119221575,0.0,1.0,1035.0,566.0,354.0,0.0,2348.0,1137.0,0.2366277171182634,0.0


In [60]:
categorical_features = [
    "categorical_chain_name", 
    "categorical_mall_name", 
    "categorical_plaace", 
    "categorical_sales_channel_name", 
    "categorical_street", 
    "categorical_district_name", 
    "categorical_lon_lat_clustering", 
    "categorical_municipality_name",   
    "categorical_grunnkrets_name"
]

# TRAIN

for feat in categorical_features:
    df_train[feat] = df_train[feat].asfactor()


In [61]:
# TEST

for feat in categorical_features:
    df_test[feat] = df_test[feat].asfactor()


In [62]:
y = "revenue"
x = df_train.columns
x.remove(y)

In [63]:
aml = H2OAutoML(sort_metric='rmse', max_runtime_secs=10*60, seed=1)
aml.train(x=x, y="revenue", training_frame = df_train)

AutoML progress: |
11:51:30.914: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5446415,0.0100179,0.5358027,0.557006,0.5329488,0.5491466,0.5483034
mean_residual_deviance,0.5170656,0.0148657,0.4996328,0.5322285,0.5027213,0.5235905,0.5271552
mse,0.5170656,0.0148657,0.4996328,0.5322285,0.5027213,0.5235905,0.5271552
null_deviance,2701.5957,106.74364,2664.5046,2664.135,2827.9321,2788.6108,2562.7964
r2,0.5072915,0.0190507,0.5084677,0.4839777,0.5335485,0.5154513,0.4950123
residual_deviance,1329.2843,32.929012,1309.0378,1374.7463,1318.1353,1350.8634,1293.6387
rmse,0.7190131,0.0103574,0.7068471,0.7295399,0.7090284,0.7235955,0.7260545
rmsle,0.3150184,0.0051870,0.3083959,0.3204907,0.3106850,0.3177241,0.3177962


In [64]:
lb = aml.leaderboard

In [65]:
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_4_20221110_115130,0.718907,0.516827,0.544485,0.314958,0.516827
StackedEnsemble_AllModels_4_AutoML_4_20221110_115130,0.719771,0.51807,0.544986,0.315224,0.51807
StackedEnsemble_BestOfFamily_6_AutoML_4_20221110_115130,0.719772,0.518072,0.546258,0.315229,0.518072
StackedEnsemble_AllModels_3_AutoML_4_20221110_115130,0.719883,0.518232,0.545136,0.315255,0.518232
StackedEnsemble_AllModels_2_AutoML_4_20221110_115130,0.720681,0.519382,0.546419,0.315407,0.519382
StackedEnsemble_BestOfFamily_4_AutoML_4_20221110_115130,0.720828,0.519593,0.546613,0.315583,0.519593
StackedEnsemble_BestOfFamily_3_AutoML_4_20221110_115130,0.72127,0.520231,0.547186,0.315551,0.520231
StackedEnsemble_AllModels_5_AutoML_4_20221110_115130,0.722672,0.522254,0.547273,0.315823,0.522254
StackedEnsemble_BestOfFamily_5_AutoML_4_20221110_115130,0.722733,0.522343,0.548274,0.316047,0.522343
StackedEnsemble_AllModels_1_AutoML_4_20221110_115130,0.723146,0.52294,0.547663,0.316608,0.52294


In [66]:
predictions = aml.predict(df_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\frikk\AppData\Roaming\Python\Python310\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'categorical_chain_name' has levels not trained on: ["24 7 TRENINGSSENTER", "ACE SHOPS", "AVANCIA SPORT", "BADEMILJÃ˜", "BADERINGEN", "BAROKK FRISÃ˜R", "BERTONI NORWAY", "BIRK SPORT", "BLUE ENERGY", "DESINESS", ...23 not listed..., "SALT OG PEPPER HÃ˜NEFOSS", "SKOGSTAD DETALJ", "SKY FITNESS", "TIGER OF SWEDEN", "TIPPY", "VARMEFAG INVEST", "VITA EXCLUSIVE", "YOGIS", "ZAVANNA", "ZAXIZ FRISÃ˜R"]
  warnings.warn(w)
C:\Users\frikk\AppData\Roaming\Python\Python310\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'categorical_mall_name' has levels not trained on: ["Aamodt Senter", "Bardufoss Torgsenter", "Bellevuesenteret", "BorgeÃ¥sen Senter", "CC Mart'n", "Combisenteret FiskÃ¥", "Coop Mega HÃ¸nefoss", "Coop Mega Ã˜vre Ã…rdal", "Cuben KjÃ¸pesenter", "De Syv SÃ¸stre kjÃ¸pesenter", ...37 not listed..., "StrÃ¸get", "Sundt Motehus", "Svelvik NÃ¦rsenter"

In [67]:
predictions_pd = h2o.as_list(predictions)
predictions_list = predictions_pd["predict"].tolist()
predictions_list = np.expm1(predictions_list)

In [68]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [69]:
predictions_df = pd.DataFrame()

predictions_df['id'] = stores_test_id
print(len(predictions_list))
predictions_df['predicted'] = predictions_list
predictions_df.to_csv("../predictions/h2o_automl3.csv", index=False)

8577
